# Auto Optimization

# Make sure keras-tuner in installed
!pip install keras-tuner

In [1]:
# Import our dependencies
import pandas as pd
import sklearn as skl
import tensorflow as tf

In [2]:
# Import dataset
census_df = pd.read_csv('Data/Five_Year_Data.csv')

In [3]:
# Modifying census_df to to exclude certain categorical columns
census_df = census_df.drop(columns=['GEO_ID','County'])
census_df.columns

Index(['Below_9th_grade', '9_12th_grade', 'High_School_Grad', 'Some_College',
       'Associate's_degree', 'Bachelor's_degree', 'Graduate_degree',
       'Population (16+)', 'Employed', 'Median Household Income',
       'Per capita Income', 'Unemployment Rate', 'Professional (Occupation)',
       'Service (Occupation)', 'Sales (Occupation)',
       'Construction (Occupation)', 'Production (Occupation)',
       'Salary (Class)', 'Gov't (Class)', 'Self-employed (Class)',
       'Fam-unpaid (Class)', 'Poverty', 'Child-poverty', 'Median Home value',
       'Median Mortgage', 'Median Rent', 'Total Population', 'White', 'Black',
       'American Indian', 'Asian', 'Pacific Islander', 'Hispanic/Latino',
       'State', 'Year'],
      dtype='object')

In [4]:
# Creating dummy data for categorical columns 'State' and 'Year'
census_encoded = pd.get_dummies(census_df, columns=['State','Year'], dtype='int')
census_encoded.head()

,Below_9th_grade,9_12th_grade,High_School_Grad,Some_College,Associate's_degree,Bachelor's_degree,Graduate_degree,Population (16+),Employed,Median Household Income,...,State_Virginia,State_Washington,State_West Virginia,State_Wisconsin,State_Wyoming,Year_2018,Year_2019,Year_2020,Year_2021,Year_2022
0,2.6,8.7,32.6,20.3,8.1,15.9,11.8,43368.0,24124.0,58786.0,...,0,0,0,0,0,1,0,0,0,0
1,2.7,7.0,27.6,22.0,9.4,20.7,10.6,167712.0,93379.0,55962.0,...,0,0,0,0,0,1,0,0,0,0
2,8.2,18.8,35.7,18.1,7.0,7.8,4.4,20948.0,8720.0,34186.0,...,0,0,0,0,0,1,0,0,0,0
3,5.7,11.1,47.3,18.6,5.8,7.6,3.9,18470.0,8099.0,45340.0,...,0,0,0,0,0,1,0,0,0,0
4,7.5,12.4,34.0,21.4,12.0,8.1,4.5,45915.0,21346.0,48695.0,...,0,0,0,0,0,1,0,0,0,0


In [5]:
census_encoded.columns

Index(['Below_9th_grade', '9_12th_grade', 'High_School_Grad', 'Some_College',
       'Associate's_degree', 'Bachelor's_degree', 'Graduate_degree',
       'Population (16+)', 'Employed', 'Median Household Income',
       'Per capita Income', 'Unemployment Rate', 'Professional (Occupation)',
       'Service (Occupation)', 'Sales (Occupation)',
       'Construction (Occupation)', 'Production (Occupation)',
       'Salary (Class)', 'Gov't (Class)', 'Self-employed (Class)',
       'Fam-unpaid (Class)', 'Poverty', 'Child-poverty', 'Median Home value',
       'Median Mortgage', 'Median Rent', 'Total Population', 'White', 'Black',
       'American Indian', 'Asian', 'Pacific Islander', 'Hispanic/Latino',
       'State_Alabama', 'State_Alaska', 'State_Arizona', 'State_Arkansas',
       'State_California', 'State_Colorado', 'State_Connecticut',
       'State_Delaware', 'State_District of Columbia', 'State_Florida',
       'State_Georgia', 'State_Hawaii', 'State_Idaho', 'State_Illinois',
       'S

In [6]:
census_encoded.dtypes

Below_9th_grade       float64
9_12th_grade          float64
High_School_Grad      float64
Some_College          float64
Associate's_degree    float64
                       ...   
Year_2018               int32
Year_2019               int32
Year_2020               int32
Year_2021               int32
Year_2022               int32
Length: 89, dtype: object

In [7]:
# Setting Target Variable
X = census_encoded.drop(columns=['Total Population'])
y = census_encoded['Total Population']

# Using sklearn to split dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

# Create scaler instance
X_scaler = skl.preprocessing.StandardScaler()

# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [8]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Grab number of input features for the first layer model
    number_input_features = len(X_train_scaled[0])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=number_input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [9]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

c:\Users\joshu\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.0

Best val_accuracy So Far: 0.0
Total elapsed time: 00h 00m 15s

Search: Running Trial #6

Value             |Best Value So Far |Hyperparameter
sigmoid           |relu              |activation
1                 |3                 |first_units
6                 |5                 |num_layers
3                 |9                 |units_0
7                 |1                 |units_1
9                 |1                 |units_2
3                 |1                 |units_3
7                 |1                 |units_4
3                 |3                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/3


KeyboardInterrupt: 